In [93]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch

In [94]:
inputs = pd.read_csv("iris.csv")
inputs.pop("Id")

targets = inputs.pop("Species")

inputs = np.array(inputs)
inputs_og = inputs

targets = np.array(targets)
targets_og = targets   
new_targets = []

for _ in range(len(targets)):
    if targets[_]=="Iris-setosa":
        new_targets.append([1,0,0])
    elif targets[_]=="Iris-versicolor":
        new_targets.append([0,1,0])
    else:
        new_targets.append([0,0,1])

     
targets = np.array(new_targets)



In [95]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size= 0.2, random_state=342)

print(inputs_train.shape, targets_train.shape)


(120, 4) (120, 3)


In [96]:
inputs_train = StandardScaler().fit_transform(inputs_train)
inputs_test = StandardScaler().fit_transform(inputs_test)

In [97]:
inputs_train = torch.from_numpy(inputs_train.astype(np.float32)) 
inputs_test = torch.from_numpy(inputs_test.astype(np.float32)) 
targets_train = torch.from_numpy(targets_train.astype(np.float32)) 
targets_test = torch.from_numpy(targets_test.astype(np.float32)) 

In [98]:
class IrisModel(torch.nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, inputs):
        targets_pred = torch.softmax(self.linear(inputs),dim=-1) 
        return targets_pred
        

In [99]:
#pipeline

logistic_model = IrisModel(4,3)

loss_fn = torch.nn.BCELoss()

opt = torch.optim.SGD(logistic_model.parameters(), lr=1e-2)

In [100]:
epochs = 500

for epoch in range(epochs):
    targets_pred = logistic_model(inputs_train)
    loss = loss_fn(targets_pred, targets_train)
    
    loss.backward()
    
    opt.step()
    opt.zero_grad()
    
    if (epoch+1)%10 == 0:
        
        print("Epoch: ", epoch+1, "  Loss: ", loss.item())
        
        with torch.no_grad():
        
            targets_test_pred = logistic_model(inputs_test)
            targets_test_cls = targets_test_pred.detach().clone()

            for i in range(len(targets_test_cls)):
                for j in range(len(targets_test_cls[i])):
                    if targets_test_cls[i][j] == max(targets_test_cls[i]):
                        targets_test_cls[i][j] = 1
                    else:
                        targets_test_cls[i][j] = 0
            
            targets_test_temp = targets_test.detach().clone()

            
            correct = 0
            
            for i in range(len(targets_test_temp)):
                if targets_test_temp[i].eq(targets_test_cls[i]).sum()==3:
                    correct += 1
            
            acc = correct/len(targets_test)
            accuracy = round(acc*100)
            
            print("Accuracy : ",accuracy,"%")
            
            if accuracy>90:
                break 
            

Epoch:  10   Loss:  0.749512791633606
Accuracy :  23 %
Epoch:  20   Loss:  0.7155160903930664
Accuracy :  33 %
Epoch:  30   Loss:  0.6844174861907959
Accuracy :  40 %
Epoch:  40   Loss:  0.6560032367706299
Accuracy :  40 %
Epoch:  50   Loss:  0.6300598382949829
Accuracy :  47 %
Epoch:  60   Loss:  0.6063777208328247
Accuracy :  50 %
Epoch:  70   Loss:  0.5847547054290771
Accuracy :  57 %
Epoch:  80   Loss:  0.5649980902671814
Accuracy :  63 %
Epoch:  90   Loss:  0.5469275712966919
Accuracy :  67 %
Epoch:  100   Loss:  0.5303756594657898
Accuracy :  70 %
Epoch:  110   Loss:  0.5151890516281128
Accuracy :  73 %
Epoch:  120   Loss:  0.5012282729148865
Accuracy :  77 %
Epoch:  130   Loss:  0.48836779594421387
Accuracy :  80 %
Epoch:  140   Loss:  0.47649502754211426
Accuracy :  80 %
Epoch:  150   Loss:  0.4655095338821411
Accuracy :  77 %
Epoch:  160   Loss:  0.4553220868110657
Accuracy :  80 %
Epoch:  170   Loss:  0.44585341215133667
Accuracy :  80 %
Epoch:  180   Loss:  0.437033385038375

In [120]:
index = 130

inputs_og = StandardScaler().fit_transform(inputs_og.astype(np.float32))

x = inputs_og[index].reshape(1,-1)
x = torch.from_numpy(x)

pred = logistic_model(x)

print(pred)

pred = pred.detach().numpy().reshape(-1)
ans = np.where(pred == max(pred))

if ans[0][0]==0:
    final_ans = "Iris-setosa"
elif ans[0][0]==1:
    final_ans = "Iris-versicolor"
else:
    final_ans = "Iris-virginica"

print("Prediction: ",final_ans, "  Target: ",targets_og[index])



tensor([[0.8257, 0.1619, 0.0124]], grad_fn=<SoftmaxBackward>)
Prediction:  Iris-setosa   Target:  Iris-setosa
